In [15]:
import networkx as nx
import numpy as np
import d3networkx as d3nx

In [23]:
G = d3nx.d3graph.D3Graph(nx.read_weighted_edgelist("data/path.edgelist"))

In [27]:
d3 = await d3nx.create_d3nx_visualizer()
d3.clear()
d3.set_graph(G)
d3.update()

websocket server started...

networkx connected...visualizer connected...

In [42]:
import networkx as nx
import heapq
from asyncio import sleep
from typing import Union
import more_itertools


async def dijkstra(
    graph: Union[nx.Graph, nx.DiGraph], source, target
) -> tuple[list[int], float]:
    """
    Find the shortest path between two nodes in a weighted graph using Dijkstra's algorithm.

    Args:
        graph: The input graph represented as a NetworkX Graph object.
        source: The index of the source node.
        target: The index of the target node.

    Returns:
        A tuple containing the shortest path as a list of node IDs and its total length.
    """
    # Determine if the graph is directed or undirected
    directed = isinstance(graph, nx.DiGraph)

    # Initialize the distances of all nodes to infinity
    dist = {node: float("inf") for node in graph.nodes()}
    dist[source] = 0

    d3.clear_highlights()
    d3.highlight_nodes((source,))
    d3.update()

    # Initialize the heap queue with the source node
    heap = [(0, source)]

    # Initialize the predecessor dictionary
    pred = {}

    while heap:
        # Pop the node with the minimum distance from the heap queue
        curr_dist, curr_node = heapq.heappop(heap)

        # If we've reached the target node, terminate early
        if curr_node == target:
            break

        # Iterate over the neighbors of the current node
        neighbors = (
            graph.successors(curr_node) if directed else graph.neighbors(curr_node)
        )
        for neighbor in neighbors:
            # Calculate the distance from the current node to the neighbor
            edge_weight = graph[curr_node][neighbor]["weight"]
            neighbor_dist = curr_dist + edge_weight

            # Update the distance and predecessor of the neighbor if a shorter path was found
            if neighbor_dist < dist[neighbor]:
                dist[neighbor] = neighbor_dist
                pred[neighbor] = curr_node

                # Add the neighbor to the heap queue
                heapq.heappush(heap, (neighbor_dist, neighbor))
        
            d3.highlight_nodes((neighbor,))
        d3.update()
        await sleep(0.5)

    # Construct the shortest path from the predecessor dictionary
    path = [target]
    while path[-1] != source:
        path.append(pred[path[-1]])
    path.reverse()

    d3.clear_highlights()
    d3.highlight_nodes(path)
    d3.highlight_edges(more_itertools.pairwise(path))
    d3.update()

    # Return the shortest path and its total length
    return path, dist[target]

In [43]:
await dijkstra(G, "S", "E")

(['S', 'B', 'H', 'G', 'E'], 7.0)

In [44]:
d3.clear_highlights()
d3.update()